<a href="https://colab.research.google.com/github/palit-ishan/Object_Detection_Project/blob/main/Road_Signs_Detection_Model_Training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Train a custom object detection model with TensorFlow Lite Model Maker

We will use the [TensorFlow Lite Model Maker](https://www.tensorflow.org/lite/guide/model_maker) to train a custom object detection model to detect Road Signs and put the TFLite model on a Raspberry Pi.

The Model Maker library uses *transfer learning* to simplify the process of training a TensorFlow Lite model using a custom dataset. Retraining a TensorFlow Lite model with your own custom dataset reduces the amount of training data required and will shorten the training time.


## Preparation

### Install the required packages
We will install the required packages, including the Model Maker package from the [GitHub repo](https://github.com/tensorflow/examples/tree/master/tensorflow_examples/lite/model_maker) and the pycocotools library we will use for evaluation.

In [ ]:
!pip install -q tflite-model-maker
!pip install -q tflite-support

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 577.3/577.3 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 32.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 240.6/240.6 kB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.3/88.3 kB 3.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 203.8/203.8 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.8/108.8 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.3/10.3 MB 49.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.9/10.9 MB 40.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 840.9/840.9 kB 16.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 591.0/591.0 kB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.9/60.9 MB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 47.3 MB/s eta 0

Import the required packages.

In [ ]:
import numpy as np
import os

from tflite_model_maker.config import ExportFormat, QuantizationConfig
from tflite_model_maker import model_spec
from tflite_model_maker import object_detector

from tflite_support import metadata

import tensorflow as tf
assert tf.__version__.startswith('2')

tf.get_logger().setLevel('ERROR')
from absl import logging
logging.set_verbosity(logging.ERROR)

/usr/local/lib/python3.9/dist-packages/tensorflow_addons/utils/ensure_tf_install.py:53: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.9.0 and strictly below 2.12.0 (nightly versions are not supported). 
 The versions of TensorFlow you are currently using is 2.8.4 and is not supported. 
Some things might work, some things might not.
If you were to encounter a bug, do not file an issue.
If you want to make sure you're using a tested and supported configuration, either change the TensorFlow version or the TensorFlow Addons's version. 
You can find the compatibility matrix in TensorFlow Addon's readme:
https://github.com/tensorflow/addons
  warnings.warn(


### Prepare the dataset

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Train the object detection model

### Step 1: Load the dataset

* Images in `train_data` is used to train the custom object detection model.
* Images in `val_data` is used to check if the model can generalize well to new images that it hasn't seen before.

In [ ]:
train_data = object_detector.DataLoader.from_pascal_voc(
    'drive/MyDrive/Project_Test_Data/train',
    'drive/MyDrive/Project_Test_Data/train',
    ['Road Work','Pedestrian Crossing','Bicycle Crossing','Stop Sign', 'Traffic Sign - Red','Traffic Sign - Green','Traffic Signal - Yellow']
)
val_data = object_detector.DataLoader.from_pascal_voc(
    'drive/MyDrive/Project_Test_Data/validate',
    'drive/MyDrive/Project_Test_Data/validate',
    ['Road Work','Pedestrian Crossing','Bicycle Crossing','Stop Sign', 'Traffic Sign - Red','Traffic Sign - Green','Traffic Signal - Yellow']
)

In [ ]:
val_data.size

7

### Step 2: Select a model architecture

EfficientDet-Lite[0-4] are a family of mobile/IoT-friendly object detection models derived from the [EfficientDet](https://arxiv.org/abs/1911.09070) architecture.

Here is the performance of each EfficientDet-Lite models compared to each others.

| Model architecture | Size(MB)* | Latency(ms)** | Average Precision*** |
|--------------------|-----------|---------------|----------------------|
| EfficientDet-Lite0 | 4.4       | 146           | 25.69%               |
| EfficientDet-Lite1 | 5.8       | 259           | 30.55%               |
| EfficientDet-Lite2 | 7.2       | 396           | 33.97%               |
| EfficientDet-Lite3 | 11.4      | 716           | 37.70%               |
| EfficientDet-Lite4 | 19.9      | 1886          | 41.96%               |

<i> * Size of the integer quantized models. <br/>
** Latency measured on Raspberry Pi 4 using 4 threads on CPU. <br/>
*** Average Precision is the mAP (mean Average Precision) on the COCO 2017 validation dataset.
</i>

In this notebook, we use EfficientDet-Lite0 to train our model. You can choose other model architectures depending on whether speed or accuracy is more important to you.

In [ ]:
spec = model_spec.get('efficientdet_lite0')

### Step 3: Train the TensorFlow model with the training data.

* Set `epochs = 20`, which means it will go through the training dataset 20 times. You can look at the validation accuracy during training and stop when you see validation loss (`val_loss`) stop decreasing to avoid overfitting.
* Set `batch_size = 4` here so you will see that it takes 15 steps to go through the 62 images in the training dataset.
* Set `train_whole_model=True` to fine-tune the whole model instead of just training the head layer to improve accuracy. The trade-off is that it may take longer to train the model.

In [ ]:
model = object_detector.create(train_data, model_spec=spec,batch_size = 20,train_whole_model=True, epochs=20, validation_data=val_data)

Epoch 1/20
6/6 [==============================] - 48s 812ms/step - det_loss: 1.8484 - cls_loss: 1.1922 - box_loss: 0.0131 - reg_l2_loss: 0.0639 - loss: 1.9123 - learning_rate: 0.0161 - gradient_norm: 0.8444
Epoch 2/20
6/6 [==============================] - 4s 736ms/step - det_loss: 1.7620 - cls_loss: 1.1781 - box_loss: 0.0117 - reg_l2_loss: 0.0639 - loss: 1.8259 - learning_rate: 0.0246 - gradient_norm: 0.8303
Epoch 3/20
6/6 [==============================] - 6s 1s/step - det_loss: 1.6446 - cls_loss: 1.1639 - box_loss: 0.0096 - reg_l2_loss: 0.0639 - loss: 1.7085 - learning_rate: 0.0240 - gradient_norm: 0.7625
Epoch 4/20
6/6 [==============================] - 5s 852ms/step - det_loss: 1.5791 - cls_loss: 1.1419 - box_loss: 0.0087 - reg_l2_loss: 0.0639 - loss: 1.6430 - learning_rate: 0.0230 - gradient_norm: 0.8365
Epoch 5/20
6/6 [==============================] - 5s 832ms/step - det_loss: 1.5155 - cls_loss: 1.1186 - box_loss: 0.0079 - reg_l2_loss: 0.0639 - loss: 1.5794 - learning_rate: 0.0

### Step 4. Evaluate the model with the validation data.

After training the object detection model using the images in the training dataset, use the 10 images in the validation dataset to evaluate how the model performs against new data it has never seen before.

As the default batch size is 64, it will take 1 step to go through the 10 images in the validation dataset.

The evaluation metrics are same as [COCO](https://cocodataset.org/#detection-eval).

In [ ]:
val_data.size

7

In [ ]:
model.evaluate(val_data)

1/1 [==============================] - 6s 6s/step



{'AP': 0.2234066,
 'AP50': 0.3797619,
 'AP75': 0.20833333,
 'APs': -1.0,
 'APm': -1.0,
 'APl': 0.22345237,
 'ARmax1': 0.4,
 'ARmax10': 0.5285714,
 'ARmax100': 0.5285714,
 'ARs': -1.0,
 'ARm': -1.0,
 'ARl': 0.5285714,
 'AP_/Road Work': 0.4,
 'AP_/Pedestrian Crossing': 0.0,
 'AP_/Bicycle Crossing': 0.3,
 'AP_/Stop Sign': 0.1,
 'AP_/Traffic Sign - Red': 0.5,
 'AP_/Traffic Sign - Green': 0.063846156,
 'AP_/Traffic Signal - Yellow': 0.2}

### Step 5: Export as a TensorFlow Lite model.

Export the trained object detection model to the TensorFlow Lite format by specifying which folder you want to export the quantized model to. The default post-training quantization technique is [full integer quantization](https://www.tensorflow.org/lite/performance/post_training_integer_quant). This allows the TensorFlow Lite model to be smaller, run faster on Raspberry Pi CPU and also compatible with the Google Coral EdgeTPU.

In [ ]:
model.export(export_dir='.', tflite_filename='road_signs.tflite')

### Step 6:  Evaluate the TensorFlow Lite model.

Several factors can affect the model accuracy when exporting to TFLite:
* [Quantization](https://www.tensorflow.org/lite/performance/model_optimization) helps shrinking the model size by 4 times at the expense of some accuracy drop.
* The original TensorFlow model uses per-class [non-max supression (NMS)](https://www.coursera.org/lecture/convolutional-neural-networks/non-max-suppression-dvrjH) for post-processing, while the TFLite model uses global NMS that's much faster but less accurate.
Keras outputs maximum 100 detections while tflite outputs maximum 25 detections.

Therefore you'll have to evaluate the exported TFLite model and compare its accuracy with the original TensorFlow model.

In [ ]:
model.evaluate_tflite('road_signs.tflite', val_data)

7/7 [==============================] - 21s 3s/step



{'AP': 0.07539683,
 'AP50': 0.2777778,
 'AP75': 0.0,
 'APs': -1.0,
 'APm': -1.0,
 'APl': 0.07539683,
 'ARmax1': 0.08571429,
 'ARmax10': 0.114285715,
 'ARmax100': 0.114285715,
 'ARs': -1.0,
 'ARm': -1.0,
 'ARl': 0.114285715,
 'AP_/Road Work': 0.05,
 'AP_/Pedestrian Crossing': 0.0,
 'AP_/Bicycle Crossing': 0.0,
 'AP_/Stop Sign': 0.011111111,
 'AP_/Traffic Sign - Red': 0.0,
 'AP_/Traffic Sign - Green': 0.4,
 'AP_/Traffic Signal - Yellow': 0.06666667}

In [ ]:
# Download the TFLite model to your local computer.
from google.colab import files
files.download('road_signs.tflite')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

**Note:** When training the model using a custom dataset, beware that if your dataset includes more than 20 classes, you'll probably have slower inference speeds compared to if you have fewer classes. This is due to an aspect of the EfficientDet architecture in which a certain layer cannot compile for the Edge TPU when it carries more than 20 classes.

Before compiling the `.tflite` file for the Edge TPU, it's important to consider whether your model will fit into the Edge TPU memory. 

The Edge TPU has approximately 8 MB of SRAM for [caching model paramaters](https://coral.ai/docs/edgetpu/compiler/#parameter-data-caching), so any model close to or over 8 MB will not fit onto the Edge TPU memory. That means the inference times are longer, because some model parameters must be fetched from the host system memory.

One way to elimiate the extra latency is to use [model pipelining](https://coral.ai/docs/edgetpu/pipeline/), which splits the model into segments that can run on separate Edge TPUs in series. This can significantly reduce the latency for big models.

The following table provides recommendations for the number of Edge TPUs to use with each EfficientDet-Lite model.

| Model architecture | Minimum TPUs | Recommended TPUs
|--------------------|-------|-------|
| EfficientDet-Lite0 | 1     | 1     |
| EfficientDet-Lite1 | 1     | 1     |
| EfficientDet-Lite2 | 1     | 2     |
| EfficientDet-Lite3 | 2     | 2     |
| EfficientDet-Lite4 | 2     | 3     |

If you need extra Edge TPUs for your model, then update `NUMBER_OF_TPUS` here: